In [1]:
import osmnx as ox
import pandas as pd
import folium
import matplotlib
import numpy as np

## Sports Fields, Parks, and Schools

In [2]:
place = "Missoula County, Montana"
tags = {"leisure":["pitch", "park", "sports_centre"], 
        'building':'school', 
        'amenity':'school'}
mso_features_gdf = ox.features.features_from_place(place, tags)

the result is a GeoDataFrame
- WGS84 crs

In [3]:
print(mso_features_gdf.shape)
print(type(mso_features_gdf))
print(mso_features_gdf.crs)

(519, 62)
<class 'geopandas.geodataframe.GeoDataFrame'>
epsg:4326


In [4]:
mso_features_gdf.head()

geometry leisure  \
element id                                               
node    357933624  POINT (-113.99518 46.84717)    park   
        357936716  POINT (-113.98729 46.86618)    park   
        357936751  POINT (-113.82621 46.89159)    park   
        357937671  POINT (-113.92455 46.82687)    park   
        357938771  POINT (-113.99872 46.87507)     NaN   

                                              name sport access manufacturer  \
element id                                                                     
node    357933624                    Campbell Park   NaN    NaN          NaN   
        357936716                     Madison Park   NaN    NaN          NaN   
        357936751           Marco Flat Picnic Area   NaN    NaN          NaN   
        357937671        Pattee Canyon Picnic Area   NaN    NaN          NaN   
        357938771  Saint Josephs Elementary School   NaN    NaN          NaN   

                  surface amenity   ele gnis:feature_id  ...  ref courts park  \
element id                                               ...                    
node    357933624     NaN     NaN   976          780870  ...  NaN    NaN  NaN   
        357936716     NaN     NaN   977          786768  ...  NaN    NaN  NaN   
        357936751     NaN     NaN  1020          786833  ...  NaN    NaN  NaN   
        357937671     NaN     NaN  1259          788695  ...  NaN    NaN  NaN   
        357938771     NaN  school   977          789931  ...  NaN    NaN  NaN   

                  not:brand:wikidata tourism type addr:country isced:level  \
element id                                                                   
node    357933624                NaN     NaN  NaN          NaN         NaN   
        357936716                NaN     NaN  NaN          NaN         NaN   
        357936751                NaN     NaN  NaN          NaN         NaN   
        357937671                NaN     NaN  NaN          NaN         NaN   
        357938771                NaN     NaN  NaN          NaN         NaN   

                  landuse nohousenumber  
element id                               
node    357933624     NaN           NaN  
        357936716     NaN           NaN  
        357936751     NaN           NaN  
        357937671     NaN           NaN  
        357938771     NaN           NaN  

[5 rows x 62 columns]

## Separate features

In [7]:
mso_features_gdf['leisure'].value_counts()

leisure
pitch            264
park             164
sports_centre     10
Name: count, dtype: int64

### For parks, pitches, and sports centers (count 264):
- 'park', 'pitch', or 'sports_centre' in ['leisure'] (case-insensitive)

In [3]:
mso_parks = mso_features_gdf[mso_features_gdf['leisure'] == 'park']
mso_pitches = mso_features_gdf[mso_features_gdf['leisure'] == 'pitch']
mso_sports_centres = mso_features_gdf[mso_features_gdf['leisure'] == 'sports_centre']

### For schools (count 81)
- 'school' in ["name"], ["building"], or ["amenity"] (case-sensitive)

In [4]:
mso_schools = mso_features_gdf[mso_features_gdf.apply(lambda x: True if "school" in [str(x['amenity']).lower(), str(x['building']).lower(), str(x['name']).lower()] else False, axis=1)]

### Save to disk (for analysis in qgis)
- use EPSG 2256 Montana State Plane (ft)
- https://epsg.io/2256

In [8]:
for gdf_obj, layer_name in [(mso_parks, "parks"), (mso_pitches, "pitches"), 
                            (mso_sports_centres, "sports_centres"), (mso_schools, "schools")]:

    # convert to a feet-based crs 
    gdf_obj.to_crs(epsg=2256, inplace=True)

    # save as layer in a GeoPackage
    gdf_obj.to_file("mso_features.gpk", layer=layer_name, driver="GPKG")

/home/vince/miniconda3/envs/ox/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: The filename extension should be 'gpkg' instead of 'gpk' to conform to the GPKG specification.
  ogr_write(


## Parking areas

In [87]:
place = "Missoula, Montana"
tags = {"parking":True, 'amenity':'parking'}
parking_gdf = ox.features.features_from_place(place, tags)
parking_gdf.shape

(2705, 34)

In [137]:
parking_gdf['parking'].value_counts()

parking
surface         112
multi-storey      5
street_side       2
underground       1
Name: count, dtype: int64

In [167]:
# almost every instance is a polygon

parking_gdf['geometry'].apply(lambda x: str(x).split("(")[0]).value_counts()

geometry
POLYGON          2703
POINT               1
MULTIPOLYGON        1
Name: count, dtype: int64

## Plot interactively with folium

In [ ]:
from branca.element import Template, MacroElement

m = folium.Map(location=[sports_gdf.geometry.centroid.y.mean(), sports_gdf.geometry.centroid.x.mean()], zoom_start=15, tiles=None)

# Add two (2) basemap layers
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='ESRI Aerial Imagery',
    overlay=False,
    control=True
).add_to(m)

folium.TileLayer('OpenStreetMap', name = "OSM Basemap").add_to(m)

# assign colors dynamically
types = [e for e in sports_gdf['leisure'].unique() if pd.notna(e)] + ["school", "parking"]  # skip nan\
colors = matplotlib.cm.get_cmap('viridis', len(types))
color_dict = {t:matplotlib.colors.rgb2hex(colors(i)) for i, t in enumerate(types)}

# Create feature groups for each type and add to map
feature_groups = {}
for t in types:
    feature_groups[t] = folium.FeatureGroup(name=t).add_to(m)

# Function to add polygons to the map
def add_polygons(gdf, map_object):
    for _, row in gdf.iterrows():
        
        # assign type and color
        if 'leisure' in row.keys() and pd.notna(row['leisure']):
            item_type = row['leisure']
            curr_color = color_dict[row['leisure']]
        elif pd.notna(row['amenity']) and row['amenity'] == 'parking':
            item_type='parking'
            curr_color = color_dict['parking']
        else:
            item_type = 'school'
            curr_color = color_dict['school']

        if pd.notna(row['name']):
            popup_name = row['name']
        else:
            popup_name = ""
        
        # python's late binding behavior with lambdas!
        sim_geo = folium.GeoJson(row.geometry, 
                                style_function=lambda x, color=curr_color: {'fillColor': color, 'color': color}, control=False)
        
        # Check for non-null and convert name to string
        #if pd.notna(row['name']):
        #    folium.Popup(str(row['name'])).add_to(sim_geo)

        folium.Popup(f"<strong>{item_type.capitalize()}</strong> \n\n {popup_name.capitalize()}").add_to(sim_geo)

        sim_geo.add_to(feature_groups[item_type])

# Function to add points to the map
# these are all schools
def add_points(gdf, map_object):

    for _, row in gdf.iterrows():

        # assign type and color
        if 'leisure' in row.keys() and pd.notna(row['leisure']):
            item_type = row['leisure']
            curr_color = color_dict[row['leisure']]
        else:
            item_type = 'school'
            curr_color = color_dict['school']

        if pd.notna(row['name']):
            popup_name = row['name']
        else:
            popup_name = ""

        folium.CircleMarker(
            [row.geometry.y, row.geometry.x],
            popup=f"<strong>{item_type.capitalize()}</strong> \n\n {popup_name.capitalize()}",  # Assuming you have a 'name' column for the popup
            color=curr_color, 
            fill=True, 
            fill_color=curr_color,
            fill_opacity=0.6,
            radius=4,
            control=False
        ).add_to(feature_groups[item_type])

# add the data to the map
add_polygons(sports_gdf[sports_gdf.geometry.type == 'Polygon'], m)
add_points(sports_gdf[sports_gdf.geometry.type == 'Point'], m)
add_polygons(parking_gdf[parking_gdf.geometry.type == 'Polygon'], m)


# add a legend
template = """
{% macro html(this, kwargs) %}
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: auto; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white; padding: 5px;">
    <p><strong>Legend</strong></p>
    {% for key, color in this.color_dict.items() %}
    <p><span style="color: {{color}}">&#11044;</span> {{key}}</p>
    {% endfor %}
</div>
{% endmacro %}
"""

macro = MacroElement()
macro._template = Template(template)
macro.color_dict = color_dict
m.get_root().add_child(macro)

# add a title
map_title = "OpenStreetMap Parks and School Features (Missoula County, MT | 01-30-2024)"
title_html = f'<h1 style="position:absolute;z-index:100000;left:40vw" ><strong>{map_title}</strong></h1>'
m.get_root().html.add_child(folium.Element(title_html))

folium.LayerControl(collapsed=False, position="bottomright").add_to(m)

# Display the map
display(m)
m.save("OSM_sports-schools-parking.html")

In [ ]:
# Create a map object centered on the mean of your data's coordinates
m = folium.Map(location=[parking_gdf.geometry.centroid.y.mean(), parking_gdf.geometry.centroid.x.mean()], zoom_start=15)

# Function to add polygons to the map
def add_polygons(gdf, map_object):
    for _, row in gdf.iterrows():
        
        # python's late binding behavior with lambdas!
        sim_geo = folium.GeoJson(row.geometry, 
                                style_function=lambda x: {'fillColor': "purple", 'color': "purple"})
        
        # Check for non-null and convert name to string
        if pd.notna(row['name']):
            folium.Popup(str(row['name'])).add_to(sim_geo)
        sim_geo.add_to(map_object)

# Function to add points to the map
def add_points(gdf, map_object):
    for _, row in gdf.iterrows():
        # Assuming geometry is a point
        folium.Marker(
            [row.geometry.y, row.geometry.x],
            popup=row['name'],  # Assuming you have a 'name' column for the popup
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(map_object)

# add a title
map_title = "OpenStreetMap Parking Features (Missoula County, MT | 01-30-2024)"
title_html = f'<h1 style="position:absolute;z-index:100000;left:40vw" ><strong>{map_title}</strong></h1>'
m.get_root().html.add_child(folium.Element(title_html))

# Call the function with polygons or points
add_polygons(parking_gdf[parking_gdf.geometry.type == 'Polygon'], m)
add_points(parking_gdf[parking_gdf.geometry.type == 'Point'], m)

# Display the map
display(m)
m.save("osm_parking.html")

## Pitches on School Grounds

1. Pitches on School Grounds

In [5]:
joined = mso_pitches.sjoin(mso_schools, how='left', predicate='intersects')

# flags for grouping pitches based on school proximity
# account for intersecting with multiple schools
school_intersect_series = joined.groupby(joined.index)['id_right'].apply(lambda x: x.notna().any())
mso_pitches['school_intersects'] = mso_pitches.index.map(school_intersect_series)

# list of intersecting school names (handles multiple matches with a concatenated list)
school_name_intersects = joined.groupby(joined.index)['name_right'].apply(lambda x: ",".join(map(str,x[x.notna()])))
mso_pitches['school_intersects_Names'] = mso_pitches.index.map(school_name_intersects)
mso_pitches['school_intersects_Names'].replace("", np.nan, inplace=True) 

/home/vince/miniconda3/envs/ox/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/vince/miniconda3/envs/ox/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/tmp/ipykernel_196302/3898312318.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The 

2. Pitches within 200 ft of school grounds

In [6]:
# convert CRS
mso_pitches_ft = mso_pitches.to_crs(epsg=2256)
mso_schools_ft = mso_schools.to_crs(epsg=2256)

buffer_distance = 200 

# create buffer around pitches
mso_pitches_buffered = mso_pitches_ft.copy()
mso_pitches_buffered['geometry'] = mso_pitches_ft.geometry.buffer(buffer_distance)

# performa spatial join
joined = mso_pitches_buffered.sjoin(mso_schools_ft, how='left', predicate='intersects')

# flags for grouping pitches based on school proximity
# account for intersecting with multiple schools
school_200ft_series = joined.groupby(joined.index)['id_right'].apply(lambda x: x.notna().any())
mso_pitches['school_200ft'] = mso_pitches.index.map(school_200ft_series)

# list of intersecting school names (handles multiple matches with a concatenated list)
school_name_200ft = joined.groupby(joined.index)['name_right'].apply(lambda x: ",".join(map(str,x[x.notna()])))
mso_pitches['school_200ft_Names'] = mso_pitches.index.map(school_name_200ft)
mso_pitches['school_200ft_Names'].replace("", np.nan, inplace=True) 

# remove duplicate classification
mso_pitches['school_200ft'] = mso_pitches.apply(lambda x: True if x['school_200ft'] and not x['school_intersects'] else False, axis =1)
mso_pitches['school_200ft_Names'] = mso_pitches['school_200ft_Names'].mask(mso_pitches['school_200ft'] == False, np.nan)

/home/vince/miniconda3/envs/ox/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/vince/miniconda3/envs/ox/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/tmp/ipykernel_196302/521317683.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The b

3. Pitches Not Associated with Schools

In [7]:
mso_pitches['school_NotAssociated'] = mso_pitches.apply(lambda x: True if x['school_intersects'] == False and x['school_200ft'] == False else False, axis = 1)

/home/vince/miniconda3/envs/ox/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [12]:
def map_sports_to_schools(sports_gdf):
    m = folium.Map(location=[sports_gdf.geometry.centroid.y.mean(), sports_gdf.geometry.centroid.x.mean()], zoom_start=15, 
                        tiles=None)

    # Add two (2) basemap layers
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='ESRI Aerial Imagery',
        overlay=False,
        control=True
    ).add_to(m)

    folium.TileLayer('OpenStreetMap', name = "OSM Basemap", overlay = False, control=True).add_to(m)

    def popup_string(row):
        # line format: "<strong>{item_type.capitalize()}: </strong> {popup_name.capitalize()} \n"
        result = "" 
        elements = []
        # add type of pitch
        elements.append(("Name", row['name'] if pd.notna(row['name']) else ""))
        elements.append(("Sport", row['sport'] if pd.notna(row['sport']) else "")) 
        elements.append(("School(s)", row['school_intersects_Names'] if row['school_intersects'] else row['school_200ft_Names'] if row['school_200ft'] else ""))

        for item in elements:
            result += f"<strong>{item[0]}: </strong> {item[1]} \n\n"
        
        return result
        
    # Function to add polygons to the map
    def add_polygons(gdf, map_object):
        poly_group = folium.FeatureGroup(name="Polygons", overlay=True, control=False).add_to(map_object)
        for _, row in gdf.iterrows():

            #is_school = True if pd.notna(row['id_right']) else False

            # python's late binding behavior with lambdas!
            sim_geo = folium.GeoJson(row.geometry, 
                                    style_function=lambda x, school_intersects=row['school_intersects'], school_200ft=row['school_200ft']: 
                                                                {'fillColor': "green", 'color': "green"} if school_intersects
                                                                else {'fillColor': "blue", 'color': "blue"} if school_200ft 
                                                                else {'fillColor': "red", 'color': "red"})
            
            popup = popup_string(row)
            folium.Popup(popup).add_to(sim_geo)
            sim_geo.add_to(poly_group)

    # Function to add points to the map
    def add_points(gdf, map_object):
        point_group = folium.FeatureGroup(name="Points", overlay=True, control=False).add_to(map_object)
        for _, row in gdf.iterrows():
            #is_school = True if pd.notna(row['id_right']) else False

            folium.Marker(
                [row.geometry.y, row.geometry.x],
                popup=popup_string(row),  # Assuming you have a 'name' column for the popup
                icon=folium.Icon(color= 'green' if row['school_intersects'] else 'blue' if row['school_200ft'] else 'red', icon='info-sign')
            ).add_to(point_group)

    # add a title
    map_title = "Pitches Associated with School (Missoula County, MT | 01-30-2024)"
    title_html = f'<h1 style="position:absolute;z-index:100000;left:40vw" ><strong>{map_title}</strong></h1>'
    m.get_root().html.add_child(folium.Element(title_html))

    # Call the function with polygons or points
    add_polygons(sports_gdf[sports_gdf.geometry.type == 'Polygon'], m)
    add_points(sports_gdf[sports_gdf.geometry.type == 'Point'], m)

    # Add a legend to the map
    legend_html = '''
    <div style="
        position: fixed; 
        bottom: 50px; left: 50px; width: 200px; height: 170px; 
        background-color: white; z-index: 1000; 
        font-size:14px; padding: 10px; 
        border-radius: 5px;
        box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
    ">
        <strong>Legend</strong><br>
        <span style="display: inline-block; width: 12px; height: 12px; background-color: green; border-radius: 50%; margin-right: 5px;"></span>
        Pitches On School Premises<br>
        <span style="display: inline-block; width: 12px; height: 12px; background-color: blue; border-radius: 50%; margin-right: 5px;"></span>
        Pitches Directly Adjacent to School<br>
        <span style="display: inline-block; width: 12px; height: 12px; background-color: red; border-radius: 50%; margin-right: 5px;"></span>
        Pitches Not Associated With a School
    </div>
    '''

    m.get_root().html.add_child(folium.Element(legend_html))

    folium.LayerControl(collapsed=False, position="bottomright",).add_to(m)

    # Display the map
    display(m)
    
    m.save("OSM_pitch-classifcation_1.html")



In [ ]:
map_sports_to_schools(mso_pitches)

New Section 


In [ ]:
print("hello world")

In [ ]:
print("hello world, again!  ")